## Assignment 2 | Problem 1
The MNIST dataset can be directly imported from trusted cloud platforms and verified entries such as `tensorflow.org`. This data can then be loaded and preprocessed before classification training and testing.

Feature structure as described by TensorFlow:
```
FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})
```

Therefore, the label feature will be discarded as it is not required for the classification task.

In [1]:
import certifi
import numpy as np
import os
from tensorflow.keras.datasets import mnist

# Force HTTPS to use specific certificate for MNIST installation
os.environ['SSL_CERT_FILE'] = certifi.where()

# Load and preprocess MNIST
(x_train, _), (x_test, _) = mnist.load_data()

Neural networks perform better with small input values, where a range of 0-1 is better than a range of 0-255. Large input values cause large activations, which makes large gradients, and leads to unstable weight updates. As a result, the input values will be normalised to maintain a more accurate output.

The type conversion to float32 is to allow decimals for increased precision when normalising, where float32 is preferred over float64 for computational speed and hardware optimisation.

A channel value of 1 (for greyscale, as opposed to 3 for RGB) is appended at the end of the input vector to conform with the 4D vector requirements for a CNN.

In [2]:
# Normalise and add channel dimension
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [3]:
# Create rotation labels (0: none, 1: left, 2: right)
def apply_rotations(images):
    n = len(images)
    rotated = []
    labels = []

    # Split into thirds
    split = n // 3
    # No rotation
    rotated.append(images[:split])
    labels.extend([0] * split)
    # Left rotation (90° CCW)
    rotated.append(np.rot90(images[split:2*split], k=1, axes=(1, 2)))
    labels.extend([1] * split)
    # Right rotation (90° CW)
    rotated.append(np.rot90(images[2*split:3*split], k=-1, axes=(1, 2)))
    labels.extend([2] * split)

    return np.concatenate(rotated), np.array(labels)

In [4]:
x_train_rot, y_train_rot = apply_rotations(x_train)
x_test_rot, y_test_rot = apply_rotations(x_test)

In [5]:
# Split training into train/validation (70/30)
val_split = int(0.7 * len(x_train_rot))
x_val = x_train_rot[val_split:]
y_val = y_train_rot[val_split:]
x_train_final = x_train_rot[:val_split]
y_train_final = y_train_rot[:val_split]